Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [2]:
import pandas as pd
import geopandas
cross_lochs = pd.read_csv('cross_lochs_sample_for_rf2.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf


/home/rzw/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
0,617,0.000951,0.005150,0.005988,0.006720,0.007361,0.010606,0.012082,0.014234,0.015797,...,0.097626,0.096350,0.095506,0.098056,0.096859,618,POINT (444764.308 6472590.655),0.000951,calluna_mix,4
1,822,0.002991,0.004075,0.004819,0.005989,0.007388,0.009729,0.012035,0.013280,0.016029,...,0.091884,0.090096,0.090060,0.089917,0.090350,823,POINT (444774.308 6472585.655),0.002991,calluna_mix,5
2,1230,0.002812,0.003526,0.005374,0.006186,0.007536,0.009726,0.011938,0.013003,0.016773,...,0.087527,0.090578,0.089190,0.087724,0.090253,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix,7
3,1841,0.002776,0.004215,0.004849,0.006305,0.008160,0.010193,0.012259,0.014886,0.015938,...,0.093998,0.096585,0.099090,0.097692,0.098034,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix,10
4,2452,0.003292,0.003483,0.003686,0.005693,0.007814,0.009010,0.012300,0.013145,0.015174,...,0.094432,0.094272,0.097161,0.097323,0.095702,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,19455,0.000811,0.001748,0.002514,0.003086,0.002629,0.003547,0.005152,0.006167,0.008431,...,0.010550,0.012407,0.010226,0.010765,0.010614,19456,POINT (445574.308 6472130.655),0.000811,water,96
176,21481,0.000730,0.001129,0.001588,0.002542,0.002014,0.003243,0.004675,0.004775,0.006566,...,0.005632,0.006227,0.007418,0.005301,0.004491,21482,POINT (445554.308 6472080.655),0.000730,water,106
177,22294,0.000828,0.001481,0.002596,0.002506,0.002702,0.003921,0.005196,0.006644,0.008155,...,0.010491,0.014097,0.011939,0.011159,0.008924,22295,POINT (445559.308 6472060.655),0.000828,water,110
178,41439,0.000852,0.001206,0.003087,0.003662,0.005048,0.007325,0.009386,0.011490,0.013876,...,0.080829,0.080322,0.081170,0.080854,0.080922,41440,POINT (444859.308 6471585.655),0.000852,calluna_mix,205


In [14]:
# cross_lochs_vis = cross_lochs_gdf[cross_lochs_gdf.columns[7:66],[359:363]] # issues with the second set of square brackets

cross_lochs_vis_bands = cross_lochs_gdf.iloc[:,6:66]
cross_lochs_RE_bands = cross_lochs_gdf.iloc[:,62:76]
cross_lochs_NIR_bands = cross_lochs_gdf.iloc[:,66:186]
cross_lochs_SWIR_bands = cross_lochs_gdf.iloc[:,186:359]

cross_lochs_general = cross_lochs_gdf.iloc[:,359:363]

cross_lochs_vis = pd.concat([cross_lochs_vis_bands, cross_lochs_general], axis=1)
cross_lochs_RE = pd.concat([cross_lochs_RE_bands, cross_lochs_general], axis=1)
cross_lochs_NIR = pd.concat([cross_lochs_NIR_bands, cross_lochs_general], axis = 1)
cross_lochs_SWIR = pd.concat([cross_lochs_SWIR_bands, cross_lochs_general], axis = 1)


cross_lochs_RE.head()

,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,...,band60,band61,band62,band63,band64,band65,POINTID,geometry,GRID_CODE,PFT
0,0.010606,0.012082,0.014234,0.015797,0.017999,0.020951,0.022017,0.024078,0.025434,0.026861,...,0.071062,0.073167,0.078536,0.085416,0.095944,0.107573,618,POINT (444764.308 6472590.655),0.000951,calluna_mix
1,0.009729,0.012035,0.013280,0.016029,0.017933,0.019669,0.021410,0.024057,0.025453,0.026926,...,0.068943,0.070627,0.076062,0.082774,0.092966,0.105358,823,POINT (444774.308 6472585.655),0.002991,calluna_mix
2,0.009726,0.011938,0.013003,0.016773,0.018120,0.019991,0.021654,0.022294,0.024200,0.025544,...,0.067762,0.069605,0.074502,0.081670,0.090580,0.101656,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix
3,0.010193,0.012259,0.014886,0.015938,0.017736,0.019947,0.021650,0.024211,0.025308,0.027024,...,0.069476,0.070655,0.075528,0.082384,0.090955,0.100739,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix
4,0.009010,0.012300,0.013145,0.015174,0.016987,0.019356,0.020710,0.021810,0.024471,0.025492,...,0.070235,0.072349,0.078022,0.085424,0.097117,0.108545,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix


In [15]:
from sklearn.model_selection import train_test_split

train_shrub_sphagnum, test_shrub_sphagnum = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('shrub_sphagnum')], test_size=0.3)
train_water, test_water = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('water')], test_size=0.3)
train_sphagnum_r, test_sphagnum_r = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('sphagnum_r')], test_size=0.3)
train_pool_bogbean, test_pool_bogbean = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('pool_bogbean')], test_size=0.3)
train_calluna_mix, test_calluna_mix = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('calluna_mix')], test_size=0.3)
train_rushes_sedges, test_rushes_sedges = train_test_split(cross_lochs_vis[cross_lochs_vis['PFT'].str.contains('rushes_sedges')], test_size=0.3)

train_frames = [train_shrub_sphagnum, train_water, train_sphagnum_r, train_pool_bogbean, train_calluna_mix, train_rushes_sedges]
test_frames = [test_shrub_sphagnum, test_water, test_sphagnum_r, test_pool_bogbean, test_calluna_mix, test_rushes_sedges]


train = pd.concat(train_frames)
test = pd.concat(test_frames)


train.head()

,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,...,band60,band61,band62,band63,band64,band65,POINTID,geometry,GRID_CODE,PFT
108,0.010163,0.011890,0.013837,0.016314,0.018191,0.020919,0.022640,0.024136,0.027162,0.028997,...,0.071509,0.072734,0.077970,0.085039,0.094314,0.107160,26747,POINT (445489.308 6471950.655),0.000941,shrub_sphagnum
92,0.009493,0.011113,0.013169,0.014782,0.018057,0.019313,0.021710,0.023569,0.025453,0.026692,...,0.071078,0.073136,0.077456,0.085615,0.095159,0.108089,11852,POINT (445109.308 6472315.655),0.001311,shrub_sphagnum
94,0.009578,0.011863,0.013655,0.015159,0.018074,0.019688,0.021585,0.023554,0.025455,0.027616,...,0.069144,0.072297,0.075954,0.083554,0.094792,0.106281,13278,POINT (445134.308 6472280.655),0.001527,shrub_sphagnum
113,0.009189,0.010890,0.012489,0.014805,0.017204,0.019284,0.020895,0.022826,0.024598,0.026318,...,0.067376,0.068804,0.073232,0.079488,0.088908,0.097538,35854,POINT (445349.308 6471725.655),0.000354,shrub_sphagnum
110,0.006544,0.008679,0.010375,0.012820,0.014858,0.016606,0.018694,0.021576,0.022686,0.024934,...,0.062110,0.065024,0.068370,0.075736,0.085246,0.098448,32950,POINT (445039.308 6471795.655),-0.000345,shrub_sphagnum


In [21]:
x_train = train[train.columns[0:60]]
y_train = train[train.columns[63]]

x_test = test[test.columns[0:60]]
y_test = test[test.columns[63]]

x_test.head(2)

,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,...,band56,band57,band58,band59,band60,band61,band62,band63,band64,band65
93,0.008793,0.011263,0.012536,0.015310,0.017103,0.018998,0.021293,0.022683,0.024858,0.026729,...,0.068228,0.067402,0.066823,0.067543,0.067584,0.069275,0.073922,0.081738,0.091988,0.105040
91,0.007386,0.010119,0.009831,0.012146,0.014883,0.016672,0.019297,0.020458,0.022726,0.024412,...,0.066404,0.065610,0.065326,0.065289,0.065831,0.067893,0.071761,0.079699,0.090286,0.103807


In [22]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


Baseline Accuracy: 0.16666666666666666


In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=2, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.5793650793650794
0.51 accuracy with a standard deviation of 0.05


In [24]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

0.52 accuracy with a standard deviation of 0.03


In [27]:
model_results = test[test.columns[61:64]]

model_results.head()

,geometry,GRID_CODE,PFT
93,POINT (445114.308 6472290.655),0.001495,shrub_sphagnum
91,POINT (445009.308 6472340.655),0.000833,shrub_sphagnum
106,POINT (445124.308 6472000.655),0.001371,shrub_sphagnum
103,POINT (445269.308 6472020.655),0.001278,shrub_sphagnum
89,POINT (444954.308 6472375.655),0.001433,shrub_sphagnum


In [28]:
train_focused = train[train.columns[61:64]]

train_focused.head()

,geometry,GRID_CODE,PFT
108,POINT (445489.308 6471950.655),0.000941,shrub_sphagnum
92,POINT (445109.308 6472315.655),0.001311,shrub_sphagnum
94,POINT (445134.308 6472280.655),0.001527,shrub_sphagnum
113,POINT (445349.308 6471725.655),0.000354,shrub_sphagnum
110,POINT (445039.308 6471795.655),-0.000345,shrub_sphagnum


In [29]:
model_results.to_file("cross_lochs_results_sample_vis_gdf.shp")

In [30]:
train_focused.to_file("cross_lochs_train_sample_vis_gdf.shp")

In [31]:
model_results.to_csv("cross_lochs_results_sample_vis_gdf.csv")
train_focused.to_csv("cross_lochs_train_sample_vis_gdf.csv")

In [32]:
# now want to predict for the whole dataset using the random forest
cross_lochs = pd.read_csv('../cross_lochs_gdf.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
0,0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,...,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248,1,POINT (444724.308 6472605.655),0.001099,NaN
1,1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,...,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.308 6472605.655),0.003365,NaN
2,2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,...,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3,POINT (444734.308 6472605.655),0.002603,NaN
3,3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,...,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4,POINT (444739.308 6472605.655),0.002381,NaN
4,4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,...,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5,POINT (444744.308 6472605.655),0.002342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42016,42016,0.001845,0.002345,0.004714,0.005430,0.006393,0.008860,0.010772,0.012449,0.014887,...,0.077407,0.076553,0.078615,0.078627,0.078251,0.077265,42017,POINT (445714.308 6471575.655),0.001845,NaN
42017,42017,0.002455,0.002271,0.004029,0.005480,0.006396,0.008863,0.011025,0.012685,0.014688,...,0.073803,0.074018,0.074871,0.075158,0.073604,0.075263,42018,POINT (445719.308 6471575.655),0.002455,NaN
42018,42018,0.002387,0.002279,0.004424,0.005426,0.006352,0.008929,0.010853,0.012708,0.014964,...,0.082753,0.083324,0.083870,0.084588,0.081337,0.082178,42019,POINT (445724.308 6471575.655),0.002387,NaN
42019,42019,0.002777,0.002263,0.004564,0.005324,0.006339,0.008972,0.011173,0.012952,0.015367,...,0.094481,0.092089,0.094319,0.095786,0.090843,0.091052,42020,POINT (445729.308 6471575.655),0.002777,NaN


In [35]:
x = cross_lochs_gdf[cross_lochs_gdf.columns[6:66]]

x.head()

,band6,band7,band8,band9,band10,band11,band12,band13,band14,band15,...,band56,band57,band58,band59,band60,band61,band62,band63,band64,band65
0,0.009240,0.010558,0.011710,0.013801,0.015441,0.018356,0.018346,0.020203,0.022126,0.022933,...,0.053691,0.052325,0.051622,0.052108,0.051371,0.052743,0.056894,0.063847,0.077289,0.095265
1,0.008654,0.010603,0.012083,0.012975,0.015468,0.017522,0.017913,0.018742,0.021310,0.021882,...,0.049172,0.048398,0.047851,0.047639,0.047351,0.048963,0.052981,0.060437,0.073406,0.089232
2,0.008977,0.010990,0.012230,0.014469,0.017382,0.018761,0.020558,0.021800,0.023982,0.025146,...,0.065350,0.063262,0.062666,0.064049,0.062703,0.065323,0.070318,0.079338,0.093115,0.109797
3,0.009825,0.012374,0.014953,0.016326,0.018532,0.020183,0.022729,0.024178,0.026417,0.029186,...,0.076114,0.076191,0.075271,0.075347,0.076439,0.078568,0.083830,0.093169,0.106521,0.122714
4,0.009762,0.010999,0.012902,0.015368,0.017144,0.018765,0.019632,0.021990,0.023575,0.025346,...,0.065877,0.064973,0.064116,0.065183,0.065097,0.067878,0.072251,0.081557,0.095215,0.112238


In [36]:
clf_pred_full = clf.predict(x)

clf_pred_full

array(['rushes_sedges', 'rushes_sedges', 'calluna_mix', ...,
       'calluna_mix', 'shrub_sphagnum', 'shrub_sphagnum'], dtype=object)

In [37]:
model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[360:362]]

model_results_full = model_results_full.assign(clf_pred_full = clf_pred_full)

model_results_full.head()

,geometry,GRID_CODE,clf_pred_full
0,POINT (444724.308 6472605.655),0.001099,rushes_sedges
1,POINT (444729.308 6472605.655),0.003365,rushes_sedges
2,POINT (444734.308 6472605.655),0.002603,calluna_mix
3,POINT (444739.308 6472605.655),0.002381,shrub_sphagnum
4,POINT (444744.308 6472605.655),0.002342,calluna_mix


In [38]:
model_results_full.to_file("cross_lochs_results_sample_full_vis_gdf.shp")

model_results_full.to_csv("cross_lochs_results_sample_full_vis_gdf.csv")


/tmp/ipykernel_3535/4012539031.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  model_results_full.to_file("cross_lochs_results_sample_full_vis_gdf.shp")
